# Task 4: Exploratory Data Analysis

Relatively simple queries
Something relating to alumni of my University

In [ ]:
# Google hires most from where?
'''
match p=(u:Edu_Institution)--(n:People)--(c:Company {name: 'Google'})
with u, collect(p) as paths
return u, paths, size(paths) as path_len
order by path_len desc skip 0 limit 1;
'''

In [ ]:
# Where do Students of UC usually go?

In [1]:
# Top 10 companies that hire most DS
'''
match (c:Company)-[r:EMPLOYEE_IN]-(e)
where r.title =~ '(?i).*data.*'
with c, collect(r) as ds_roles, collect(e) as emp
with *, size(ds_roles) as ds_count
return *
order by ds_count desc limit 10;
'''